In [33]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

import time
# temp sleep
# time.sleep(3600)

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn1_glove_sample_feat_3.pkl
(159571, 6) (153164, 6)
file path ../features/cnn1_glove_sample_feat_4.pkl
(159571, 6) (153164, 6)
file path ../features/cnn2d_muse_1_sample_feat_3.pkl
(159571, 6) (153164, 6)
file path ../features/cnn2d_muse_1_sample_feat_4.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_sample_feat_3.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_sample_feat_4.pkl
(159571, 6) (153164, 6)
file path ../features/gru1_glove_sample_feat_3.pkl
(159571, 6) (153164, 6)
file path ../features/gru1_glove_sample_feat_4.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm1_glove_sample_feat_3.pkl
(159571, 6) (153164, 6)
file path ../features/lstm1_glove_sample_feat_4.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153

In [34]:
train[list_classes].describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
def aug_data_ratio(x,y,ratio=0.2):
    print(x.shape)
    neg_index = np.where(y[:,0]==0)[0]
    print(neg_index)
    data_cnt = len(neg_index)
    add_cnt = int(data_cnt*ratio)
    add_index = neg_index[:add_cnt]
    add_x = np.concatenate([x,x[add_index]])
    add_y = np.concatenate([y,y[add_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

def del_data_ratio(x,y,ratio=0.8):
    print(x.shape)
    pos_index = np.where(y[:,0]==1)[0]
    neg_index = np.where(y[:,0]==0)[0]
    print(pos_index)
    data_cnt = len(pos_index)
    add_cnt = int(data_cnt*ratio)
    add_index = pos_index[:add_cnt]
    add_x = np.concatenate([x[add_index],x[neg_index]])
    add_y = np.concatenate([y[add_index],y[neg_index]])
    print(add_x.shape,data_cnt)
    return add_x,add_y

# add neg data
#train_x,train_y = aug_data_ratio(train_x,train_y)
# del pos data
train_x,train_y = del_data_ratio(train_x,train_y)

(159571, 160)
[     6     12     16 ..., 159541 159546 159554]
(156512, 160) 15294


In [36]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cache_test_pred = np.zeros((153164,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((153164,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [37]:
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res/1.2
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold5_div2.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.648643	valid-logloss:0.648719
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[557]	train-logloss:0.060293	valid-logloss:0.071459

0
0.0594299268687 0.0714903981624
[0]	train-logloss:0.645415	valid-logloss:0.645405
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[185]	train-logloss:0.014649	valid-logloss:0.016804

1
0.0137972441057 0.016839745975
[0]	train-logloss:0.646554	valid-logloss:0.646562
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[318]	train-logloss:0.031505	valid-logloss:0.035364

2
0.0305682173934 0.0354391130724
[0]	train-logloss:0.64471	valid-logloss:0.644747
Multiple e

[0]	train-logloss:0.645408	valid-logloss:0.645483
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[189]	train-logloss:0.014628	valid-logloss:0.01754

1
0.0137520660901 0.0175896009727
[0]	train-logloss:0.646543	valid-logloss:0.646521
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[283]	train-logloss:0.031934	valid-logloss:0.03638

2
0.03100421087 0.036452241804
[0]	train-logloss:0.64471	valid-logloss:0.644742
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[213]	train-logloss:0.003912	valid-logloss:0.006621

3
0.00335768604895 0.00668727702954
[0]	train-logloss:0.647243	valid-logloss:0.647369
Multiple eva

In [40]:
from scipy.special import logit,expit

# The logit function is defined as logit(p) = log(p/(1-p)). 
# Note that logit(0) = -inf, logit(1) = inf, and logit(p) for p<0 or p>1 yields nan.

# The expit function, also known as the logistic function, 
# is defined as expit(x) = 1/(1+exp(-x)). It is the inverse of the logit function.

def new_trans(x,sub=0.5):
    return expit(logit(x)-sub)

test_x = np.array([0,0.5,0.8,1])
print(new_trans(test_x))
print(new_trans(test_x,0.4))
print(new_trans(test_x,0.6))

div_num = np.sqrt(np.e)
print(div_num)
    

[ 0.          0.37754067  0.70812487  1.        ]
[ 0.          0.40131234  0.72835537  1.        ]
[ 0.          0.35434369  0.6870351   1.        ]
1.6487212707


In [45]:
sample_submission[list_classes] = new_trans(xgb_res,0.5)
sample_submission.to_csv("../results/xgb_ens_sample_csv_fold5_new_trans.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.996803      0.215588  0.955420  0.129460  0.869520   
1  0000247867823ef7  0.000106      0.000014  0.000060  0.000010  0.000044   
2  00013b17ad220c46  0.000136      0.000017  0.000069  0.000010  0.000089   
3  00017563c3f7919a  0.000055      0.000019  0.000072  0.000016  0.000059   
4  00017695ad8997eb  0.002609      0.000025  0.000237  0.000019  0.000348   

   identity_hate  
0       0.339707  
1       0.000026  
2       0.000047  
3       0.000029  
4       0.000115  
save done
